
## IPyWidget Viewer 

I did a viewer using ipywidgets that allows to visualize the results, with the slider you can move the deepth in the z-axis


In [1]:
import ipywidgets as widgets
from ipywidgets import FloatSlider
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout
from IPython.display import display,clear_output
from IPython.display import display
import IPython
from ipywidgets import Output

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

import numpy as np
import imutils
import cv2
import pandas as pd
import SimpleITK as sitk  # For loading the dataset
import numpy as np  # For data manipulation
from model import build_model, dice_coefficient  # For creating the model
import glob  # For populating the list of files
from scipy.ndimage import zoom  # For resizing
import re  # For parsing the filenames (to know their modality)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.models import load_model

%matplotlib inline

Using TensorFlow backend.
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Fu

In [2]:
def post_process_label(img):
    kernel = np.ones((3, 3))
    imgo = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=3)
    kernel = np.ones((3, 3))
    imgo = cv2.dilate(imgo, kernel, iterations=1)
    return imgo

In [3]:
def read_img(img_path):
    """
    Reads a .nii.gz image and returns as a numpy array.
    """
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path))

def resize(img, shape, mode='constant', orig_shape=(155, 240, 240)):
    """
    Wrapper for scipy.ndimage.zoom suited for MRI images.
    """
    assert len(shape) == 3, "Can not have more than 3 dimensions"
    factors = (
        shape[0]/orig_shape[0],
        shape[1]/orig_shape[1], 
        shape[2]/orig_shape[2]
    )
    
    # Resize to the given shape
    return zoom(img, factors, mode=mode)


def preprocess(img, out_shape=None):
    """
    Preprocess the image.
    Just an example, you can add more preprocessing steps if you wish to.
    """
    if out_shape is not None:
        img = resize(img, out_shape, mode='constant')
    
    # Normalize the image
    mean = img.mean()
    std = img.std()
    return (img - mean) / std


def preprocess_label(img, out_shape=None, mode='nearest'):
    """
    Separates out the 3 labels from the segmentation provided, namely:
    GD-enhancing tumor (ET — label 4), the peritumoral edema (ED — label 2))
    and the necrotic and non-enhancing tumor core (NCR/NET — label 1)
    """
    #ncr = img == 1  # Necrotic and Non-Enhancing Tumor (NCR/NET)
    ed = img == 2  # Peritumoral Edema (ED)
    et = img == 4  # GD-enhancing Tumor (ET)
    
    if out_shape is not None:
        #ncr = resize(ncr, out_shape, mode=mode)
        ed = resize(ed, out_shape, mode=mode)
        et = resize(et, out_shape, mode=mode)

    return np.array([ed, et], dtype=np.uint8)
    

# Required to run on RXT GPU Cards

In [4]:
gpu_config = ConfigProto()
gpu_config.gpu_options.allow_growth = True
session = InteractiveSession(config=gpu_config)

# Creating a viewer

In [5]:
class viewer:
    def __init__(self, img,pred):
        self.tab = widgets.Tab()
        self.childrens = []
        self.tab.children = []
        self.tab.set_title(0, 'Segmentation')
        self.tab.set_title(1, 'Nifti Information')
        self.mri_img = img
        self.mri_prediction = pred #resize(pred,self.mri_img.pixel_array.shape)  #np.zeros(512*512).reshape((512,512))
        
        self.mri_payload = self.mri_img #np.array(self.mri_img.dataobj)#np.copy(self.mri_img.pixel_array)
        self.mri_slide = self.mri_payload[:,:,10] 
        self.mri_pred_slide = self.mri_prediction[:,:,10] 
        
        
        self.tab_segmentation()
        self.tab_dicominformation()
        self.display()
        
    def tab_segmentation(self):
        self.mri_slider = widgets.IntSlider(min=0, max=self.mri_payload.shape[2]-1, step=1,value=10,orientation='vertical',description='Deep',layout=Layout(width='150px'))
        self.out_mri = widgets.Output(layout=Layout(height='{}px'.format(self.mri_slide.shape[0]), width = '{}px'.format(self.mri_slide.shape[1]), border='None'))
        self.out_prediction = widgets.Output(layout=Layout(height='{}px'.format(self.mri_slide.shape[0]), width = '{}px'.format(self.mri_slide.shape[1]), border='None'))
        self.out_overlap = widgets.Output(layout=Layout(height='{}px'.format(self.mri_slide.shape[0]), width = '{}px'.format(self.mri_slide.shape[1]), border='None'))
        scale=4
        def update_mri_plot(value):
            self.mri_slide = self.mri_payload[:,:,value]
            fig, axes = plt.subplots()
            axes.set_title('Original MRI')
            dpi = fig.get_dpi()
            fig.set_size_inches(self.mri_slide.shape[0]/float(dpi)*scale,self.mri_slide.shape[1]/float(dpi)*scale)
            axes.imshow(self.mri_slide,cmap='bone')
            plt.show(fig)
                    
            
        def plot_overlap(value):
            fig, axes = plt.subplots()
            axes.set_title('Overlap')
            dpi = fig.get_dpi()
            fig.set_size_inches(self.mri_prediction.shape[0]/float(dpi)*scale,self.mri_prediction.shape[1]/float(dpi)*scale)
            axes.imshow(self.mri_slide+ self.mri_pred_slide*np.max(self.mri_slide))
            plt.show(fig)
            
        def plot_prediction(value):
            self.mri_pred_slide = self.mri_prediction[:,:,value]
            img = self.mri_pred_slide #resize(self.mri_prediction,self.mri_img.pixel_array.shape) 
            fig, axes = plt.subplots()
            axes.set_title('Prediction')
            dpi = fig.get_dpi()
            fig.set_size_inches(self.mri_prediction.shape[0]/float(dpi)*scale,self.mri_prediction.shape[1]/float(dpi)*scale)
            axes.imshow(self.mri_pred_slide)
            #axes.imshow(self.mri_pred_slide)
            plt.show(fig)
                    
        with self.out_mri :
            img0 = interactive(update_mri_plot, value=self.mri_slider)
            self.mri_slider.value=13
        
        
        with self.out_prediction :
            img2 = interactive(plot_prediction, value=self.mri_slider)
            self.mri_slider.value=11
        
        with self.out_overlap :
            img3 = interactive(plot_overlap, value=self.mri_slider)
            self.mri_slider.value=10
                
            
        self.hbox=widgets.HBox([img0.children[0],img0.children[1],img2.children[1],img3.children[1]])

        self.childrens.append(self.hbox) 
        
    def tab_dicominformation(self):
        pass
    def display(self):
        #self.tab.children = self.childrens
        display(self.hbox)


# Loading the pre-trained model

In [7]:

def dummy_loss(y_true, y_pred):
    return y_pred

#load_model('vaemode.h5', custom_objects={'dummy_loss':dummy_loss})
model=load_model('brast-models/model-vae.h5',custom_objects={'loss_':dummy_loss,'dice_coefficient':dice_coefficient} )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# Data Preparation for prediction (read this very carefull!)

This is a multiclass segmentation model, the input data for the model are four cubes of data then
the Input Shape is (4, 128, 128, 128), 4 channels ["t1", "t1ce", "flair", "t2"]

The output are labels 1,2,4 (this will be fixed when I train it again)

labels explanation:
* GD-enhancing tumor (ET — label 4)
* The peritumoral edema (ED — label 2)
* Non-enhancing tumor (NCR/NET — label 1)



<h1 style="color: red;"> That means that to evaluate the data of a patient in the hostpital we will need the four images ("t1", "t1ce", "flair", "t2") for a single patient</h1>



In [10]:
# raw input files
mri_file_t1    = 'brast-validation/2019/MICCAI_BraTS_2019_Data_Validation/BraTS19_CBICA_AAM_1/BraTS19_CBICA_AAM_1_t1.nii.gz'
mri_file_t1ce  = 'brast-validation/2019/MICCAI_BraTS_2019_Data_Validation/BraTS19_CBICA_AAM_1/BraTS19_CBICA_AAM_1_t1ce.nii.gz'
mri_file_flair = 'brast-validation/2019/MICCAI_BraTS_2019_Data_Validation/BraTS19_CBICA_AAM_1/BraTS19_CBICA_AAM_1_flair.nii.gz'
mri_file_t2    = 'brast-validation/2019/MICCAI_BraTS_2019_Data_Validation/BraTS19_CBICA_AAM_1/BraTS19_CBICA_AAM_1_t2.nii.gz'

# output with bias correction using itk/ants
#mri_file_t1_out    = 'output/data/valid/preprocessed/BraTS19_CBICA_AAM_1_t1_out.nii.gz'
#mri_file_t1ce_out  = 'output/data/valid/preprocessed/BraTS19_CBICA_AAM_1_t1ce_out.nii.gz'
#mri_file_flair_out = 'output/data/valid/preprocessed/BraTS19_CBICA_AAM_1_flair_out.nii.gz'
#mri_file_t2_out    = 'output/data/valid/preprocessed/BraTS19_CBICA_AAM_1_t2_out.nii.gz'

#mri_file_t1_out = mri_file_t1
#mri_file_t1ce_out = mri_file_t1ce
#mri_file_flair_out = mri_file_flair
#mri_file_t2_out = mri_file_t2

# Reading images

the pre-trained model <a href='https://github.com/ellisdg/3DUnetCNN#pre-trained-models'>https://github.com/ellisdg/3DUnetCNN#pre-trained-models</a>
was trained with a input shape of (128,128,128) for every category  ["t1", "t1ce", "flair", "t2"],
the read_image function allows to read and resize the cube of data.

The final shape for the event(Tensor) to be evaluate should be (4,128,128,128)




In [11]:
input_shape = (4, 160, 192, 128)
output_channels = 2

def read_image(mrifile,input_shape):
    return preprocess(read_img(mrifile), input_shape[1:])

mri_t1=read_image(mri_file_t1,input_shape)
mri_t1ce=read_image(mri_file_t1ce,input_shape)
mri_flair=read_image(mri_file_flair,input_shape)
mri_t2=read_image(mri_file_t2,input_shape)
#mri_label = read_image(label_file,img_shape)


#mri_t1=np.array(mri_t1.dataobj)
#mri_t1ce=np.array(mri_t1ce.dataobj)
#mri_flair=np.array(mri_flair.dataobj)
#mri_t2=np.array(mri_t2.dataobj)
#mri_label = np.array(mri_label.dataobj)

print(mri_t1.shape)
print(mri_t1ce.shape)
print(mri_flair.shape)
print(mri_t2.shape)
#print(mri_label.shape)


(160, 192, 128)
(160, 192, 128)
(160, 192, 128)
(160, 192, 128)


# Standarization
they call it normalization but it is more like a data standarization 
because they are subtranting the mean and dividing by the standar deviation  

their code snippet
```py
def normalize_data(data, mean, std):
    data -= mean[:, np.newaxis, np.newaxis, np.newaxis]
    data /= std[:, np.newaxis, np.newaxis, np.newaxis]
    return data
```

# Creating the event

The four cube of images( ['t1', 't2', 't1ce', 'flair']) in a single tensor.

Shape (4,128,128,128)

In [12]:
event=np.array([mri_t1,mri_t2,mri_t1ce,mri_flair])
print(event.shape)

(4, 160, 192, 128)


# Prediction

I am using the default parameters that they are using for training, 
you can see that in config dictionary import it from the treining module.
```
from brats.train import config
```


In [13]:
pred=model.predict(np.array([event]))

In [14]:
plabel2=pred[0,0,:,:,:]
plabel4=pred[0,1,:,:,:]

plabel2 = post_process_label(plabel2)
plabel4 = post_process_label(plabel4)


# Results Visualization 

using the viewer that I did, you can to visualize the results,
maybe you can play in the prediction cell with the parameters to get better results or you can to post process it to filter some noise.

# Results label 4 (GD-enhancing tumor)

In [15]:
viewer(mri_t1,plabel4)

# Results label 2 (The peritumoral edema)

In [16]:
viewer(mri_t1,plabel2)

# All together

In [17]:
viewer(mri_t1,plabel2+(plabel4*2))

# 3D Visualization using ipyvolume

In [18]:
import ipyvolume as ipv
import ipyvolume.pylab as p3

In [19]:
ipv.figure()
ipv.volshow(plabel2)
ipv.show()

/root/.local/lib/python3.6/site-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [20]:
ipv.figure()
ipv.volshow(plabel4)
ipv.show()

In [21]:
ipv.figure()
ipv.volshow(mri_t1*0.6+plabel4*500+plabel2*255)
ipv.show()